<h1><center><b>Laboratorio N° 02 - Utilizando Operadores de Conjuntos de Datos</b></center></h1>

<h2><b>Objetivo:</b></h2>
* Aplicar los conocimientos asimilados sobre operadores de conjuntos en SQL Server

<h2><b>Ejercicio 1: Operador UNION</b></h2>

1.1. Conectar con la BD TSQL

In [1]:
USE TSQL;
GO

Commands completed successfully.

Total execution time: 00:00:00.003

1.2. Escribir una sentencia SELECT que retorne las columnas productid y productname desde la tabla Production.Products. Filtrar los resultados  para incluir solo los productos que tienen la columna categoryid valor 4.

In [3]:
SELECT productid, productname 
FROM Production.Products 
WHERE categoryid = 4;

(10 rows affected)

Total execution time: 00:00:00.012

productid,productname
11,Product QMVUN
12,Product OSFNS
31,Product XWOXC
32,Product NUNAW
33,Product ASTMN
59,Product UKXRI
60,Product WHBYK
69,Product COAXA
71,Product MYMOI
72,Product GEEOO


1.3. Escribir la sentencia SELECT para retornar las columnas productid y productname desde la tabla Production.Products. Filtrar los resultados para incluir solo los productos que tienen un monoto total de ventas mayor a $50,000. Para el monto total de ventas, se necesitara la consultar la tabla Sales.OrderDetails y agregar todos los valores (qty * unitprice) de detalle o linea de orden por cada producto.

In [4]:
SELECT p.productid, p.productname
FROM Production.Products AS p
INNER JOIN Sales.OrderDetails AS od ON p.productid = od.productid
GROUP BY p.productid, p.productname
HAVING SUM(od.qty * od.unitprice) > 50000;

(4 rows affected)

Total execution time: 00:00:00.026

productid,productname
29,Product VJXYN
38,Product QDOMO
59,Product UKXRI
60,Product WHBYK


1.4. Escribir una consulta SELECT para traer las columnas custid y contactname de la tabla Sales.Customers. Monstrar el top 10 de clientes por monto de ventas para el periodo January 2008 y mostrar el top 10 de clientes por monto de ventas para el periodo February 2008. (Ojo: escribir dos sentencias SELECT, enlazar Sales.Customers y Sales.OrderValues, y utilizar el apropiado operador de conjunto).

In [ ]:
SELECT custid, contactname, total_sales
FROM (
    SELECT TOP 10
        C.custid,
        C.contactname,
        SUM(OD.qty * OD.unitprice) AS total_sales,
        'January 2008' AS sales_period
    FROM Sales.Customers AS C
    JOIN Sales.Orders AS O ON C.custid = O.custid
    JOIN Sales.OrderDetails AS OD ON O.orderid = OD.orderid
    WHERE O.orderdate >= '2008-01-01' AND O.orderdate < '2008-02-01'
    GROUP BY C.custid, C.contactname
    ORDER BY total_sales DESC
) AS TopJanuarySales

UNION ALL

SELECT custid, contactname, total_sales
FROM (
    SELECT TOP 10
        C.custid,
        C.contactname,
        SUM(OD.qty * OD.unitprice) AS total_sales,
        'February 2008' AS sales_period
    FROM Sales.Customers AS C
    JOIN Sales.Orders AS O ON C.custid = O.custid
    JOIN Sales.OrderDetails AS OD ON O.orderid = OD.orderid
    WHERE O.orderdate >= '2008-02-01' AND O.orderdate < '2008-03-01'
    GROUP BY C.custid, C.contactname
    ORDER BY total_sales DESC
) AS TopFebruarySales
ORDER BY total_sales DESC;

(10 rows affected)

(10 rows affected)

Total execution time: 00:00:00.037

custid,contactname
39,"Song, Lolan"
32,"Krishnan, Venky"
20,"Kane, John"
76,"Gulbis, Katrin"
71,"Navarro, Tomás"
81,"Nagel, Jean-Philippe"
65,"Moore, Michael"
63,"Veronesi, Giorgio"
55,"Egelund-Muller, Anja"
89,"Smith Jr., Ronaldo"


custid,contactname
63,"Veronesi, Giorgio"
37,"Crăciun, Ovidiu V."
65,"Moore, Michael"
20,"Kane, John"
39,"Song, Lolan"
34,"Cohen, Shy"
30,"Shabalin, Rostislav"
71,"Navarro, Tomás"
62,"Misiec, Anna"
9,"Raghav, Amritansh"


<h2><b>Ejercicio 2: Operador EXCEPT y INTERSECT</b></h2>

2.1. Escribir una sentencia SELECT para traer la columna custid de la tabla Sales.Orders table. Filtrar los resultados para incluir solo los clientes quierenes compraron mas de 20 diferentes productos (basada en la columna productid de la tabla Sales.OrderDetails).

In [7]:
SELECT o.custid
FROM Sales.Orders AS o
INNER JOIN Sales.OrderDetails AS od ON o.orderid = od.orderid
GROUP BY o.custid
HAVING COUNT(DISTINCT od.productid) > 20;

(33 rows affected)

Total execution time: 00:00:00.019

custid
4
5
7
9
10
20
24
25
30
34


2.2. Escrbir una sentencia SELECT para traer la columna custid desd la tabla the Sales.Orders. Filtrar los resultados para incluir solo los clientes del pais USA y excluir todos los clientes del resultado previo (tarea 1). (Ojo: usar el operador EXCEPT la consulta anterior).

In [8]:
SELECT DISTINCT o.custid
FROM Sales.Orders AS o
INNER JOIN Sales.Customers AS c ON o.custid = c.custid
WHERE c.country = 'USA'

EXCEPT

SELECT o.custid
FROM Sales.Orders AS o
INNER JOIN Sales.OrderDetails AS od ON o.orderid = od.orderid
GROUP BY o.custid
HAVING COUNT(DISTINCT od.productid) > 20;

(10 rows affected)

Total execution time: 00:00:00.022

custid
32
36
43
45
48
55
75
77
78
82


2.3. Escribir una sentencia SELECT para traer la columna custid de la tabla Sales.Orders. Filtrar solo los clientes quienes tienen un valor de total de ventas mayor a $10,000. Calcular el valor de ventas utilizando las columnas qty y unitprice de la tabla Sales.OrderDetails.

In [9]:
SELECT o.custid
FROM Sales.Orders AS o
INNER JOIN Sales.OrderDetails AS od ON o.orderid = od.orderid
GROUP BY o.custid
HAVING SUM(od.qty * od.unitprice) > 10000;

(41 rows affected)

Total execution time: 00:00:00.017

custid
23
46
75
9
89
72
32
35
86
63


<h2><b>Ejercicio 3: Utilizando el Operador APPLY</b></h2>

3.1. Escribir una sentencia SELECT para traer las columnas productid y productname de la tabla Production.Products. En adición, para cada producto, traer las ultimas dos filas de la tabla Sales.OrderDetails basado en el número orderid.

In [10]:
SELECT p.productid, p.productname, LastOrders.orderid
FROM Production.Products AS p
CROSS APPLY
(SELECT TOP(2) od.orderid
 FROM Sales.OrderDetails AS od
 WHERE od.productid = p.productid
 ORDER BY od.orderid DESC) AS LastOrders
ORDER BY p.productid;

(154 rows affected)

Total execution time: 00:00:00.011

productid,productname,orderid
1,Product HHYDP,11070
1,Product HHYDP,11047
2,Product RECZE,11077
2,Product RECZE,11075
3,Product IMEHJ,11077
3,Product IMEHJ,11017
4,Product KSBRM,11077
4,Product KSBRM,11000
5,Product EPEIM,11047
5,Product EPEIM,11030


3.2. Utilizar el operador CROSS APPLY y una subconsulta correlacionada. Ordenar el resultado por la columna productid.

In [11]:
SELECT p.productid, p.productname, LastOrders.orderid
FROM Production.Products AS p
CROSS APPLY
(SELECT TOP(2) orderid
 FROM Sales.OrderDetails AS od
 WHERE od.productid = p.productid
 ORDER BY orderid DESC) AS LastOrders
ORDER BY p.productid;

(154 rows affected)

Total execution time: 00:00:00.007

productid,productname,orderid
1,Product HHYDP,11070
1,Product HHYDP,11047
2,Product RECZE,11077
2,Product RECZE,11075
3,Product IMEHJ,11077
3,Product IMEHJ,11017
4,Product KSBRM,11077
4,Product KSBRM,11000
5,Product EPEIM,11047
5,Product EPEIM,11030


3.3. Ejecutar la sentencia siguiente para crear la función fnGetTop3ProductsForCustomer.

In [12]:
DROP FUNCTION IF EXISTS dbo.fnGetTop3ProductsForCustomer; 
GO 
CREATE FUNCTION dbo.fnGetTop3ProductsForCustomer (@custid AS INT) 
RETURNS TABLE AS 
RETURN 
SELECT TOP(3) d.productid, p.productname, SUM(d.qty * d.unitprice) AS totalsalesamount 
FROM Sales.Orders AS o 
INNER JOIN Sales.OrderDetails AS d ON d.orderid = o.orderid 
INNER JOIN Production.Products AS p ON p.productid = d.productid 
WHERE custid = @custid 
GROUP BY d.productid, p.productname 
ORDER BY totalsalesamount DESC;

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.004

3.4. Escribir una sentencia SELECT statement para traer las columnas custid y contactname de la tabla Sales.Customers. Utilizar el operador CROSS APPLY con la función dbo.fnGetTop3ProductsForCustomer para traer las columnas productid, productname y totalsalesamount por cada cliente.

In [13]:
SELECT c.custid, c.contactname, TopProducts.productid, TopProducts.productname, TopProducts.totalsalesamount
FROM Sales.Customers AS c
CROSS APPLY dbo.fnGetTop3ProductsForCustomer(c.custid) AS TopProducts
ORDER BY c.custid, TopProducts.totalsalesamount DESC;

(265 rows affected)

Total execution time: 00:00:00.026

custid,contactname,productid,productname,totalsalesamount
1,"Allen, Michael",63,Product ICKNK,878.00
1,"Allen, Michael",59,Product UKXRI,825.00
1,"Allen, Michael",28,Product OFBNT,775.20
2,"Hassall, Mark",72,Product GEEOO,348.00
2,"Hassall, Mark",60,Product WHBYK,340.00
2,"Hassall, Mark",32,Product NUNAW,320.00
3,"Peoples, John",11,Product QMVUN,1453.20
3,"Peoples, John",26,Product HLGZA,936.90
3,"Peoples, John",59,Product UKXRI,825.00
4,"Arndt, Torsten",20,Product QHFFP,4050.00
